<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_retrieval_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install -U transformers
!pip install faiss-cpu #有cpu和gpu版本，一般cpu版本就够用
import faiss
from transformers import AutoTokenizer,AutoModelForSequenceClassification,DataCollatorWithPadding,Trainer,TrainingArguments,pipeline
from transformers import BertForSequenceClassification, BertPreTrainedModel, BertModel
from typing import Optional
from transformers.configuration_utils import PretrainedConfig
import torch
import pandas as pd
from torch.nn import CosineSimilarity, CosineEmbeddingLoss

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.6 MB/s eta 0:00:00


In [2]:
data = pd.read_csv("./law_faq.csv")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
data.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


In [3]:
#导入sentence_similarity_vector_match的模型
class BertForVectorMatch(BertPreTrainedModel):
  def __init__(self, config):
      super().__init__(config)
      self.config = config
      self.bert = BertModel(config)
      # Initialize weights and apply final processing
      self.post_init()
  def forward(
  self,
  input_ids: Optional[torch.Tensor] = None,
  attention_mask: Optional[torch.Tensor] = None,
  token_type_ids: Optional[torch.Tensor] = None,
  position_ids: Optional[torch.Tensor] = None,
  head_mask: Optional[torch.Tensor] = None,
  inputs_embeds: Optional[torch.Tensor] = None,
  labels: Optional[torch.Tensor] = None,
  output_attentions: Optional[bool] = None,
  output_hidden_states: Optional[bool] = None,
  return_dict: Optional[bool] = None,
):
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    senA_input_ids, senB_input_ids = input_ids[:, 0], input_ids[:, 1]
    senA_attention_mask, senB_attention_mask = attention_mask[:, 0], attention_mask[:, 1]
    senA_token_type_ids, senB_token_type_ids = token_type_ids[:, 0], token_type_ids[:, 1]

    senA_outputs = self.bert(
    senA_input_ids,
    attention_mask=senA_attention_mask,
    token_type_ids=senA_token_type_ids,
    position_ids=position_ids,
    head_mask=head_mask,
    inputs_embeds=inputs_embeds,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
)
    senB_outputs = self.bert(
            senB_input_ids,
            attention_mask=senB_attention_mask,
            token_type_ids=senB_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
    senA_pooled_output = senA_outputs[1]
    senB_pooled_output = senB_outputs[1]
    cos = CosineSimilarity()(senA_pooled_output, senB_pooled_output)
    loss = None
    if labels is not None:
      loss_fct = CosineEmbeddingLoss(0.3) #余弦距离<0.3就不算Loss
      loss = loss_fct(senA_pooled_output, senB_pooled_output, labels)
      #labels要么是1，要么是-1，代表同类/不同类

    output = (cos,)
    return ((loss,) + output) if loss is not None else output
model = BertForVectorMatch.from_pretrained("hfl/chinese-macbert-base")
model = model.cuda()
model.eval()

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForVectorMatch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [4]:
questions = data["title"].tolist()
print(questions)
vectors = []
with torch.inference_mode():  #有的时候模型训练没有爆显存，但是推理的时候爆了，就可以用这个
  for i in range(0,len(questions),32): #批量处理 batchsize=32
    batch_questions = questions[i:i+32]
    inputs = tokenizer(batch_questions,return_tensors="pt", padding=True, max_length=128, truncation=True)
    inputs = {k:v.to(model.device)for k,v in inputs.items()}
    vector = model.bert(**inputs)[1]
    vectors.append(vector)

vectors #vectors是tensor


['在法律中定金与订金的区别订金和定金哪个受', '盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体', '非法微整形机构构成非法经营罪吗', '入室持刀行凶伤人能不能判刑', '对交通事故责任认定书不服怎么办，交通事故损', '喝完就又去唱歌回家时去了车祸去唱歌我同意了车是我开的', '二审一般要多久', '欠钱没有欠条有录音能起诉吗', '从合伙经营公司撤资，只给欠条有用吗', '哪些情况可以先行刑事拘留', '是一个撞了人的不陪偿', '在工地上面出了安全事故怎么来维护自己的权', '申请和受理劳动争议调解仲裁时间上有什么限制', '商标注册有什么相对禁止的条件', '高利贷还不还不知如何是好。', '公司破产孕妇怎么赔偿', '两个人打架,因为对方先打我,后来找着打他的最后对方叫来五六人把我的肋骨打断', '客户拖欠货款', '到法院起诉一个人，要多少费用', '签了购房合同等于取得房屋产权吗', '迷信诈骗团伙老板和骨干会被判刑多少年', '婚后买房，婆婆出了一半钱，离婚房怎么分', '刑事案件，怎么收费，请律师都是有哪些费用', '我是个大车司机，老板拖欠工资我用什么方法能', '交通事故对方主责我次责车辆修理费怎么算', '你好，花了别人偷来送给你的钱，在公安局画押教罚款会不会案底', '请问，父亲生前欠下债务要还吗', '起诉欠款人的可行法律流程', '公司拖欠工资打什么电话投诉才可以', '结婚二+多年没在一起算是自动离婚不', '钱站利息算高利贷吗？', '你好，在公安局的笔录上画了押，还教了罚款会不会有案底', '假如房产证和户口本同时给租赁人办理营业执照会发生什么后果？', '请问我是二婚，房子是婚前买的，上午办的结婚证，下午去加的老婆名字，这样算不算夫妻共同财产', '是轻伤，他现在被拘留了还是拘留几天就放了', '如何区分贪污和受贿？', '公司法人变更工程代建合同是否还有效', '带未满周岁孩子无经济来源的签订担保有效吗', '女方欠男方的钱，离婚协议书要不要写让她归还？', '购房合同丢了怎么办', '新房与效果图不一致可否退房', '公司有权没收手机吗？', '通*亚*拖*工*工*', '老板拖欠工资拒绝给押金怎么办', '您好，我现在是监外执行期间，信用卡欠了6000千多4个月未还，被银行起诉了，现在还上还会被收监吗', '公司实行大小周

[tensor([[ 0.0203, -0.7161,  0.8741,  ..., -0.7680, -0.9453, -0.4818],
         [-0.0022, -0.7082,  0.9638,  ..., -0.8746, -0.9365, -0.9135],
         [ 0.9240, -0.5462,  0.9998,  ..., -0.9621, -0.9968, -0.8310],
         ...,
         [ 0.1400, -0.7153,  0.9948,  ..., -0.9152, -0.9887, -0.2599],
         [ 0.5367, -0.2016,  0.9999,  ..., -0.9408, -0.9949, -0.7942],
         [ 0.9608,  0.8158,  0.9995,  ..., -0.9899, -0.9940,  0.2712]],
        device='cuda:0'),
 tensor([[ 0.9587, -0.6278,  0.9999,  ..., -0.9601, -0.9954, -0.7889],
         [ 0.6596,  0.6183,  0.9940,  ..., -0.9861, -0.9853,  0.0748],
         [ 0.6214, -0.0452,  0.9989,  ..., -0.9573, -0.9764, -0.6108],
         ...,
         [-0.4009, -0.9567,  0.9280,  ..., -0.6138, -0.9230, -0.8606],
         [ 0.9973,  0.7788,  1.0000,  ..., -0.9996, -0.9999, -0.0664],
         [-0.1932, -0.4530,  0.9925,  ..., -0.5847, -0.9785, -0.7846]],
        device='cuda:0'),
 tensor([[ 0.9849,  0.5764,  1.0000,  ..., -0.9915, -0.9938, -0.75

In [5]:
len(vectors)

570

In [6]:
vectors2 = torch.concat(vectors).cpu().numpy()  #将570个1维张量连接成一个更长的1维张量。
vectors2

array([[ 0.02027395, -0.7160855 ,  0.8741444 , ..., -0.76800513,
        -0.9452624 , -0.48184586],
       [-0.00221087, -0.7081643 ,  0.9638291 , ..., -0.8745695 ,
        -0.9365493 , -0.91348267],
       [ 0.9239994 , -0.54617345,  0.9998173 , ..., -0.9620941 ,
        -0.99683666, -0.8309597 ],
       ...,
       [ 0.95313776,  0.37414235,  0.9998436 , ..., -0.98248696,
        -0.997622  , -0.52408105],
       [ 0.72103846,  0.5724356 ,  0.8890166 , ..., -0.7663223 ,
        -0.9377349 , -0.205901  ],
       [ 0.03385595, -0.44725323,  0.999465  , ..., -0.84156096,
        -0.9899933 , -0.6873971 ]], dtype=float32)

In [7]:
len(vectors2)

18213

In [8]:
vectors3 = torch.concat(vectors,dim=0).cpu().numpy()
vectors3

array([[ 0.02027395, -0.7160855 ,  0.8741444 , ..., -0.76800513,
        -0.9452624 , -0.48184586],
       [-0.00221087, -0.7081643 ,  0.9638291 , ..., -0.8745695 ,
        -0.9365493 , -0.91348267],
       [ 0.9239994 , -0.54617345,  0.9998173 , ..., -0.9620941 ,
        -0.99683666, -0.8309597 ],
       ...,
       [ 0.95313776,  0.37414235,  0.9998436 , ..., -0.98248696,
        -0.997622  , -0.52408105],
       [ 0.72103846,  0.5724356 ,  0.8890166 , ..., -0.7663223 ,
        -0.9377349 , -0.205901  ],
       [ 0.03385595, -0.44725323,  0.999465  , ..., -0.84156096,
        -0.9899933 , -0.6873971 ]], dtype=float32)

In [9]:
vectors3.shape

(18213, 768)

In [10]:
len(vectors3)

18213

In [11]:
# vectors = torch.concat(vectors, dim=0).cpu().numpy() #原本vectors是tensor,faiss是cpu版本所以要转成numpy
index = faiss.IndexFlatIP(768) #768是向量维度，vecros的shape是(18213, 768)
faiss.normalize_L2(vectors3) #vectors是对data里面的question进行tokenized的结果
index.add(vectors3) #截至至此，索引库构建成功

In [12]:
query = "寻衅滋事"
with torch.inference_mode():
  inputs = tokenizer(query, return_tensors="pt", padding=True, max_length=128, truncation=True)
  inputs = {k: v.to(model.device) for k, v in inputs.items()}
  vector = model.bert(**inputs)[1]
  q_vector = vector.cpu().numpy()
q_vector.shape


(1, 768)

In [13]:
faiss.normalize_L2(q_vector) #q_vector是要从库里面匹配需求向量
scores, indexes = index.search(q_vector, 10) #10是topk
topk_result = data.values[indexes[0].tolist()] #从库里面获取匹配到的top10结果，返回的结果是data里对应索引的全部内容，而不只是vectors

In [14]:
topk_result

array([['涉嫌寻衅滋事',
        '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等

In [15]:
topk_result[:,0]

array(['涉嫌寻衅滋事', '恶意诋毁他人', '交通事故逃逸', '交通事故肇事逃逸', '醉酒驾驶证', '监视居住条件',
       '是金融诈骗还是非法集资', '西*区抓酒驾', '哪些违章会造成终身禁驾', '飞达暴力催收'], dtype=object)

In [16]:
topk_result[:,1]

array(['说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打他人的;4

In [17]:
topk_result[0]

array(['涉嫌寻衅滋事',
       '说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重

In [19]:
# sentence_similarity_model=BertForSequenceClassification.from_pretrained("hfl/chinese-macbert-base",num_labels=1)
sentence_similarity_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/nlp/cross_model/checkpoint-30",num_labels=1)
#必须有num_labels=1，不然就变成了分类模型而不是回归模型
sentence_similarity_model=sentence_similarity_model.cuda()
sentence_similarity_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [20]:
candidates = topk_result[:,0].tolist()
querys = [query] * len(candidates)
inputs = tokenizer(querys,candidates, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(sentence_similarity_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = sentence_similarity_model(**inputs).logits
logits


tensor([[0.0574],
        [0.2110],
        [0.1889],
        [0.0987],
        [0.1612],
        [0.1348],
        [0.1971],
        [0.1440],
        [0.1475],
        [0.0621]], device='cuda:0')

In [21]:
logits.size()

torch.Size([10, 1])

In [22]:
logits=logits.squeeze() #用于移除大小为 1 的维度
logits

tensor([0.0574, 0.2110, 0.1889, 0.0987, 0.1612, 0.1348, 0.1971, 0.1440, 0.1475,
        0.0621], device='cuda:0')

In [23]:
logits.size()

torch.Size([10])

In [24]:
result = torch.argmax(logits, dim=-1)
result

tensor(1, device='cuda:0')

In [39]:
# canidate = topk_result[:, 0].tolist()
# ques = [query] * len(canidate)
# inputs = tokenizer(ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True)
# inputs = {k: v.to(sentence_similarity_model.device) for k, v in inputs.items()}
# with torch.inference_mode():
#     logits = sentence_similarity_model(**inputs).logits.squeeze()
#     result = torch.argmax(logits, dim=-1)
# result

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [25]:
candidate_answers = topk_result[:, 1].tolist()
match_quesiton = candidates[result.item()]
final_answer = candidate_answers[result.item()]
match_quesiton, final_answer

('恶意诋毁他人',
 '有下列行为之一的，处五日以下拘留或者五百元以下罚款；情节较重的，处五日以上十日以下拘留，可以并处五百元以下罚款：（一）写恐吓信或者以其他方法威胁他人人身安全的；（二）公然侮辱他人或者捏造事实诽谤他人的；（三）捏造事实诬告陷害他人，企图使他人受到刑事追究或者受到治安管理处罚的；（四）对证人及其近亲属进行威胁、侮辱、殴打或者打击报复的；（五）多次发送淫秽、侮辱、恐吓或者其他信息，干扰他人正常生活的；（六）偷窥、偷拍、窃听、散布他人隐私的。')